In [1]:
import pandas as pd
from functions import *
import plotly.express as px


# Get Data (Whether downloaded or not)

In [2]:
year = 2023
month = 1

In [3]:

path = download_data(year,month)

In [4]:
df = pd.read_parquet(path)

# Part 3 

In [5]:

def avg_fare_day(df: pd.DataFrame, date_col: str, month: int):
    df[date_col] = pd.to_datetime(df[date_col])
    df['month_num'] = df[date_col].dt.month
    unique_months = df['month_num'].unique()
    df_month = df[df['month_num'] == month].copy()
    # Ensure we are selecting available month in the data
    if df_month.shape[0]==0:
        print(f"You have entered a month that doesn't exist in the data. The available months are {', '.join(map(str, df['month_num'].unique()))}")
    else:
        # Include relevant payment options only
        df_month = df_month[(df_month['payment_type'] == 1) | (df_month['payment_type'] == 2) | (df_month['payment_type'] == 5)].copy()
        df_month['day_str'] = df_month[date_col].dt.day_name()
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        df_month['day_str'] = pd.Categorical(df_month['day_str'], categories=day_order, ordered=True)
        avg_fare = df_month.groupby('day_str', observed=False)['fare_amount'].mean().reset_index()
        avg_fare = avg_fare.sort_values('day_str')
    
        return avg_fare

In [6]:
avg_fare_report = avg_fare_day(df,'tpep_pickup_datetime',month)

In [7]:
avg_fare_report

,day_str,fare_amount
0,Monday,19.607860
1,Tuesday,18.495019
2,Wednesday,18.014310
3,Thursday,18.348330
4,Friday,18.241604
5,Saturday,17.514210
6,Sunday,19.746509


In [8]:
avg_fare_report.to_csv('average_fare_per_day.csv')

# Part 4

In [9]:
df_revenue = df[(df['payment_type'] == 1) | (df['payment_type'] == 2) | (df['payment_type'] == 5)].copy()
df_revenue['pickup_date'] = df_revenue['tpep_pickup_datetime'].dt.date
df_revenue['revenue'] = df_revenue['fare_amount'] + df_revenue['extra'] + df_revenue['improvement_surcharge'] + df_revenue['congestion_surcharge']

In [10]:
revenue_by_day  = df_revenue.groupby('pickup_date')['revenue'].sum().reset_index()
revenue_by_day['pickup_date'] = pd.to_datetime(revenue_by_day['pickup_date'])

In [11]:
revenue_by_day

,pickup_date,revenue
0,2008-12-31,73.50
1,2022-10-24,298.70
2,2022-10-25,366.10
3,2022-12-31,529.70
4,2023-01-01,1898717.19
5,2023-01-02,1693538.41
6,2023-01-03,2076539.69
7,2023-01-04,2217909.84
8,2023-01-05,2290510.21
9,2023-01-06,2272341.30


### We can see dates that are not part of January. Thus, we should filter non-January dates 

In [12]:
start_of_month, end_of_month = get_month_start_end(year,month)

In [13]:
revenue_by_day = revenue_by_day[(revenue_by_day['pickup_date']>=start_of_month) & (revenue_by_day['pickup_date']<=end_of_month)].reset_index(drop=True)

In [14]:
revenue_by_day

,pickup_date,revenue
0,2023-01-01,1898717.19
1,2023-01-02,1693538.41
2,2023-01-03,2076539.69
3,2023-01-04,2217909.84
4,2023-01-05,2290510.21
5,2023-01-06,2272341.30
6,2023-01-07,2238496.47
7,2023-01-08,2000284.64
8,2023-01-09,1975502.03
9,2023-01-10,2203958.30


In [15]:
fig = px.line(revenue_by_day, x='pickup_date', y='revenue', title='Total Revenue Per Day',
              labels={'pickup_date': 'Date', 'revenue': 'Total Revenue'},
              markers=True)

fig.show()